# DSC540 - Milestone2: Cleaning/Formatting Flat File Source
By: Janine Par

In [7]:
import numpy as np  
import pandas as pd  
import json
import sys
import json
import datetime
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 800)


# Yelp Business Data - Contains business data including location data, attributes, and categories.



## Step1: Read business json dataset

In [8]:
bus = pd.read_json("yelp_academic_dataset_business.json", lines=True)
bus.columns.values

array(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'], dtype=object)

In [9]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}"
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'CoatCheck': 'False', 'RestaurantsTakeOut': 'False', 'RestaurantsDelivery': 'False', 'Caters': 'False', 'WiFi': 'u'no'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'WheelchairAccessible': 'True', 'HappyHour': 'False', 'OutdoorSeating': 'False', 'HasTV': 'False', 'RestaurantsReservations': 'False', 'DogsAllowed': 'False', 'ByAppointmentOnly': 'False'}","Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}"
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}"
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}"


In [10]:
bus.shape

(150346, 14)

## Step 2: Clean categories that are null then only get the Restaurant and any categories that has Food. Using str and query functionality in python

In [11]:
#Filter out null categories then get only categories like restraurant, food and bar
bus= bus[bus['categories'].notnull()]
bus_rest = bus.query('categories.str.contains("Restaurants") or categories.str.contains("Food")', engine='python')
bus_rest.categories
bus_rest.shape

(64616, 14)

## Step 3: Identify fields with missing values 

In [12]:
# Check for records with missing values
for c in bus_rest.columns:
    miss=bus_rest[c].isnull().sum()
    if miss > 0:
        print ("{} has {} missing values (s)". format (c, miss))
    else:
        print ("{} has no missing values".format(c))

business_id has no missing values
name has no missing values
address has no missing values
city has no missing values
state has no missing values
postal_code has no missing values
latitude has no missing values
longitude has no missing values
stars has no missing values
review_count has no missing values
is_open has no missing values
attributes has 876 missing values (s)
categories has no missing values
hours has 9335 missing values (s)


## Step 3: Restaurant that are missing values on attribute and hours open can impact the result set when doing analysis assuming all restaurant must have a feature and have been open at least few days in a week. with that, I'm cleaning up all that do not have attribute features or hours open in the dataframe

In [13]:
# Remove records with Null attributes and hours in business restaurants
bus_rest_clean= bus_rest[~bus_rest['attributes'].isnull()]
bus_rest_clean= bus_rest_clean[~bus_rest_clean['hours'].isnull()]


In [14]:
# Check for records with missing values 
for c in bus_rest_clean.columns:
    miss=bus_rest_clean[c].isnull().sum()
    if miss > 0:
        print ("{} has {} missing values (s)". format (c, miss))
    else:
        print ("{} has no missing values".format(c))
        



business_id has no missing values
name has no missing values
address has no missing values
city has no missing values
state has no missing values
postal_code has no missing values
latitude has no missing values
longitude has no missing values
stars has no missing values
review_count has no missing values
is_open has no missing values
attributes has no missing values
categories has no missing values
hours has no missing values


### 4. Get the number of Features or Attribute that is true for the establishment

1. Business attributes on Yelp show services and amenities offered, as well as other details that are important to many consumers when selecting a business to patronize. 
2. This column is a dictionary and I think it is important to identify all values where attribute/feature = True and with that, I am creating a new column with this count information

In [7]:
bus_rest_clean['attributes'].head(4)

3                                                                                                                                                                                                                                                                                         {'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}
4                                                                                                                                                                                                                                                                                                                                                   

This section of the code creates a new column that counts the business attribute that are set to True for a business record
1. For loop reads each attribute row in the dataframe
2. Since the attribute column is a dictionary. I have created a nested loop that will read each key:value pair and if the value of the key (row[i]) is True, then increment the att_count
3. Add the generated att_count as a column in the dataframe

In [15]:
attributes_count = []
for idx, row in bus_rest_clean['attributes'].iteritems():
    att_count = 0
    for i in row:
        if row[i] == "True":
            att_count = att_count + 1
            #print(att_count)
    attributes_count.append(att_count)
bus_rest_clean['attribute_count']=attributes_count    

In [16]:
bus_rest_clean.head(4)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,attribute_count
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}",3
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}",5
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'OutdoorSeating': 'True', 'RestaurantsReservations': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'Alcohol': 'u'none'', 'Ambience': 'None', 'GoodForKids': 'True', 'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'CoatCheck': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'True', 'WiFi': 'u'no'', 'WheelchairAccessible': 'True', 'HasTV': 'True', 'HappyHour': 'False', 'DriveThru': 'True', 'BikeParking': 'False'}","Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturday': '9:0-22:0', 'Sunday': '8:0-22:0'}",9
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'RestaurantsGoodForGroups': 'False', 'BusinessAcceptsCreditCards': 'True', 'OutdoorSeating': 'True', 'GoodForKids': 'True', 'Alcohol': 'u'none'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'ByAppointmentOnly': 'False', 'WiFi': 'u'no'', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'HasTV': 'True', 'RestaurantsDelivery': 'True', 'CoatCheck': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'DriveThru': 'True', 'HappyHour': 'False', 'WheelchairAccessible': 'True'}","Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', 'Wednesday': '6:0-21:0', 'Thursday': '6:0-16:0', 'Friday': '6:0-16:0', 'Saturday': '6:0-17:0', 'Sunday': '6:0-21:0'}",8


### 5. Get Price Range from the attribute dictionary. I think price range is important in analyzing the business dataset. This approach is also can be done if there are other attributes that we would need to extract out  the dictionary and make it a dataframe column

This section of the code creates a new column that defines the value of the RestaurantsPriceRange2 which the number of dollar sign of each business (1 = $ ;2-$$ ; 3 - $$$) that represents the restaurant price range.

1. For loop reads each attribute row in the dataframe
2. The attribute column is a dictionary, we read row.keys and if any matches to RestaurantsPriceRange2 then get the value of the key and append the the list
3. Append None, if key (RestaurantsPriceRange2)  is not present in the dictionary 

In [17]:
RestaurantsPriceRange2=[]
for idx, row in bus_rest_clean['attributes'].iteritems():
        l=row.keys()
        if 'RestaurantsPriceRange2' in l:
            RestaurantsPriceRange2.append(row['RestaurantsPriceRange2'])
        else:
            RestaurantsPriceRange2.append(None)
     
bus_rest_clean['RestaurantsPriceRange2']=RestaurantsPriceRange2

In [18]:
bus_rest_clean['RestaurantsPriceRange2'].head(30)

3        1
4     None
5        1
9        1
11    None
12       1
14       1
15       2
19       1
20       2
22       2
23       2
27    None
28       2
29       1
31       1
33       1
35       2
41       2
45       2
46       2
47       2
49    None
52       2
53       1
56       2
58       2
59       1
60       2
61       2
Name: RestaurantsPriceRange2, dtype: object

## Step 6 Create a function that will generate another category of price: Low, Medium, High which depends on the (RestaurantsPriceRange2) value

1. Create a function that accepts a price value from the dataframe RestaurantsPriceRange2, which will assign a category for each of the values
2. Call the function using Apply and create another column in the dataframe with the return value

In [19]:
def price_category (price):
    if price =="1":
        return "Low"
    elif price =="2" or price =="3":
        return "Medium"
    elif price is None:
        return None
    else: 
        return "High"

In [20]:
bus_rest_clean['PriceCategory'] =bus_rest_clean['RestaurantsPriceRange2'].apply(price_category)
bus_rest_clean[['PriceCategory', 'RestaurantsPriceRange2']].head(30)

,PriceCategory,RestaurantsPriceRange2
3,Low,1
4,None,None
5,Low,1
9,Low,1
11,None,None
12,Low,1
14,Low,1
15,Medium,2
19,Low,1
20,Medium,2


# CHECK-IN DATA - This contains business checkin 
## Step1: Read another jason file provided by yelp that has check-in information for each business dataset

Examine that the check-in date all together in one column and comma separated

In [21]:
df_checkin = pd.read_json("yelp_academic_dataset_checkin.json", lines=True)
print(df_checkin.columns.values)

['business_id' 'date']


In [14]:
df_checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020-07-24 22:42:27, 2020-10-24 21:36:13, 2020-12-09 21:23:33, 2021-01-20 17:34:57, 2021-04-30 21:02:03, 2021-05-25 21:16:54, 2021-08-06 21:08:08, 2021-10-02 15:15:42, 2021-11-11 16:23:50"
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011-07-18 22:30:31, 2012-09-07 20:28:50, 2013-03-27 15:57:36, 2013-08-13 00:31:34, 2013-08-13 00:31:48, 2013-09-23 17:39:38, 2013-11-18 06:34:08, 2014-04-12 23:04:47"
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012-03-11 10:30:02, 2012-04-24 07:07:59, 2012-04-24 07:43:31, 2013-05-25 16:41:10, 2014-05-02 15:49:55, 2014-09-18 02:28:23, 2014-11-10 15:16:43, 2015-09-27 13:18:32"
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014-09-30 14:41:47, 2014-10-23 18:22:28, 2015-04-27 19:55:00, 2015-09-21 12:52:09, 2015-10-01 12:46:16, 2015-10-22 13:35:04, 2016-01-14 12:27:43, 2016-02-01 15:15:07, 2016-02-11 12:22:47, 2016-03-31 23:15:46, 2016-04-11 13:11:34, 2016-05-25 12:44:11, 2016-06-27 15:46:11, 2016-06-30 15:14:28, 2016-07-28 14:15:59, 2016-09-22 20:05:06, 2016-11-16 19:00:11, 2016-12-27 14:36:14, 2017-01-24 20:18:16, 2017-02-21 16:01:49, 2017-03-21 14:08:44, 2017-05-21 16:07:21, 2017-08-07 14:17:39, 2021-06-21 19:59:50"


## Step2: Count the number of checkin date from the date fields which is comma delimited using str.count then create a column in dataframe

In [22]:
df_checkin['checkin_count'] = df_checkin.date.str.count(',') 

#Validate if correct 
df_checkin[df_checkin.business_id=='--7jw19RH9JKXgFohspgQw']    

,business_id,date,checkin_count
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014-09-30 14:41:47, 2014-10-23 18:22:28, 2015-04-27 19:55:00, 2015-09-21 12:52:09, 2015-10-01 12:46:16, 2015-10-22 13:35:04, 2016-01-14 12:27:43, 2016-02-01 15:15:07, 2016-02-11 12:22:47, 2016-03-31 23:15:46, 2016-04-11 13:11:34, 2016-05-25 12:44:11, 2016-06-27 15:46:11, 2016-06-30 15:14:28, 2016-07-28 14:15:59, 2016-09-22 20:05:06, 2016-11-16 19:00:11, 2016-12-27 14:36:14, 2017-01-24 20:18:16, 2017-02-21 16:01:49, 2017-03-21 14:08:44, 2017-05-21 16:07:21, 2017-08-07 14:17:39, 2021-06-21 19:59:50",25


## Step 3: Join the data to the business restaurant clean dataset

In [23]:
bus_rest_merge= bus_rest_clean.join(df_checkin.set_index('business_id'), on='business_id', how='inner')
bus_rest_merge.shape

bus_rest_merge.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,attribute_count,RestaurantsPriceRange2,PriceCategory,date,checkin_count
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}",3,1,Low,"2010-08-18 17:05:36, 2010-11-25 17:45:31, 2011-02-27 16:58:20, 2011-05-07 14:53:45, 2011-07-23 14:42:36, 2011-09-10 00:47:25, 2011-09-10 22:48:43, 2011-11-24 15:08:04, 2011-12-04 17:12:11, 2011-12-11 17:42:30, 2012-01-07 01:55:08, 2012-01-27 17:25:35, 2012-02-02 23:27:25, 2012-02-08 02:42:30, 2012-02-12 02:12:58, 2012-02-19 20:00:18, 2012-03-01 21:38:08, 2012-03-03 01:36:16, 2012-03-04 19:02:10, 2012-03-11 21:45:52, 2012-03-27 22:43:44, 2012-03-31 22:49:05, 2012-04-17 22:28:54, 2012-04-21 21:23:45, 2012-06-17 21:59:17, 2012-07-07 21:11:44, 2012-07-10 20:41:38, 2012-07-13 00:16:29, 2012-07-15 16:47:10, 2012-07-19 18:19:28, 2012-07-22 20:14:42, 2012-07-29 15:35:48, 2012-08-15 18:05:50, 2012-08-23 23:39:12, 2012-08-29 11:11:15, 2012-09-08 14:56:13, 2012-09-16 15:30:07, 2012-09-17 21:54:50...",334
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}",5,None,None,"2019-06-28 21:35:09, 2019-09-12 21:07:48, 2019-11-09 23:28:44, 2020-02-15 21:10:58, 2020-08-02 16:05:42, 2020-09-12 19:47:26, 2020-11-21 17:15:26, 2020-12-05 20:55:53, 2021-03-27 16:33:48, 2021-05-01 16:10:04, 2021-06-19 18:40:25, 2021-08-20 22:20:37, 2021-09-10 22:26:40, 2021-12-17 21:55:20",13
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'OutdoorSeating': 'True', 'RestaurantsReservations': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'Alcohol': 'u'none'', 'Ambience': 'None', 'GoodForKids': 'True', 'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'CoatCheck': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'True', 'WiFi': 'u'no'', 'WheelchairAccessible': 'True', 'HasTV': 'True', 'HappyHour': 'False', 'DriveThru': 'True', 'BikeParking': 'False'}","Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturday': '9:0-22:0', 'Sunday': '8:0-22:0'}",9,1,Low,"2012-12-16 05:27:33, 2013-02-15 04:00:46, 2013-02-18 00:43:56, 2013-04-26 23:57:46, 2013-06-26 22:01:59, 2013-10-26 00:59:54, 2015-09-20 16:54:57, 2017-04-23 04:50:47, 2017-05-20 23:56:27, 2017-07-12 02:13:44, 2018-11-20 04:02:12, 2019-08-03 01:17:44, 2020-02-14 17:12:41, 2020-08-16 18:45:56, 2020-09-05 20:16:58, 2021-01-24 20:15:36, 2021-04-14 17:18:59, 2021-06-28 16:45:40, 2021-07-16 01:25:24, 2021-09-17 17:06:57, 2021-12-12 02:25:20, 2022-01-13 17:35

## Step 4: Rename the column from 'date' to 'checkin_date_list'

In [24]:
bus_rest_merge=bus_rest_merge.rename(columns={'date': 'checkin_date_list'})

bus_rest_merge.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,attribute_count,RestaurantsPriceRange2,PriceCategory,checkin_date_list,checkin_count
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}",3,1,Low,"2010-08-18 17:05:36, 2010-11-25 17:45:31, 2011-02-27 16:58:20, 2011-05-07 14:53:45, 2011-07-23 14:42:36, 2011-09-10 00:47:25, 2011-09-10 22:48:43, 2011-11-24 15:08:04, 2011-12-04 17:12:11, 2011-12-11 17:42:30, 2012-01-07 01:55:08, 2012-01-27 17:25:35, 2012-02-02 23:27:25, 2012-02-08 02:42:30, 2012-02-12 02:12:58, 2012-02-19 20:00:18, 2012-03-01 21:38:08, 2012-03-03 01:36:16, 2012-03-04 19:02:10, 2012-03-11 21:45:52, 2012-03-27 22:43:44, 2012-03-31 22:49:05, 2012-04-17 22:28:54, 2012-04-21 21:23:45, 2012-06-17 21:59:17, 2012-07-07 21:11:44, 2012-07-10 20:41:38, 2012-07-13 00:16:29, 2012-07-15 16:47:10, 2012-07-19 18:19:28, 2012-07-22 20:14:42, 2012-07-29 15:35:48, 2012-08-15 18:05:50, 2012-08-23 23:39:12, 2012-08-29 11:11:15, 2012-09-08 14:56:13, 2012-09-16 15:30:07, 2012-09-17 21:54:50...",334
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}",5,None,None,"2019-06-28 21:35:09, 2019-09-12 21:07:48, 2019-11-09 23:28:44, 2020-02-15 21:10:58, 2020-08-02 16:05:42, 2020-09-12 19:47:26, 2020-11-21 17:15:26, 2020-12-05 20:55:53, 2021-03-27 16:33:48, 2021-05-01 16:10:04, 2021-06-19 18:40:25, 2021-08-20 22:20:37, 2021-09-10 22:26:40, 2021-12-17 21:55:20",13
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'OutdoorSeating': 'True', 'RestaurantsReservations': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'Alcohol': 'u'none'', 'Ambience': 'None', 'GoodForKids': 'True', 'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'CoatCheck': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'True', 'WiFi': 'u'no'', 'WheelchairAccessible': 'True', 'HasTV': 'True', 'HappyHour': 'False', 'DriveThru': 'True', 'BikeParking': 'False'}","Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturday': '9:0-22:0', 'Sunday': '8:0-22:0'}",9,1,Low,"2012-12-16 05:27:33, 2013-02-15 04:00:46, 2013-02-18 00:43:56, 2013-04-26 23:57:46, 2013-06-26 22:01:59, 2013-10-26 00:59:54, 2015-09-20 16:54:57, 2017-04-23 04:50:47, 2017-05-20 23:56:27, 2017-07-12 02:13:44, 2018-11-20 04:02:12, 2019-08-03 01:17:44, 2020-02-14 17:12:41, 2020-08-16 18:45:56, 2020-09-05 20:16:58, 2021-01-24 20:15:36, 2021-04-14 17:18:59, 2021-06-28 16:45:40, 2021-07-16 01:25:24, 2021-09-17 17:06:57, 2021-12-12 02:25:20, 202

# TIP DATA: Tips written by a user on a business. Tips are shorter than reviews and tend to convey quick suggestions.

## Step1: Read another jason file provided by yelp that has Tips for each business dataset

In [19]:
tip = pd.read_json("yelp_academic_dataset_tip.json", lines=True)
print(tip.columns.values)

['user_id' 'business_id' 'text' 'date' 'compliment_count']


In [20]:
tip.head(1)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0


## Step 2: Get the year from the tip date. I think this information is good to have when grouping by number of tips per business each year.

In [21]:
tip['year_Tip']=pd.to_datetime(tip['date']).dt.year

tip.head()

,user_id,business_id,text,date,compliment_count,year_Tip
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0,2012
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban sandwiches,2013-02-05 18:35:10,0,2013
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0,2013
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0,2017
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0,2012


# Summary Clean-up and Transformed Datasets

1. Business Datasets Merged 

    a) Clean Business Dataset with new attributes

    b) Joined information with Check-in dataset

In [22]:
bus_rest_merge.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,attribute_count,RestaurantsPriceRange2,PriceCategory,checkin_date_list,checkin_count
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}",3,1,Low,"2010-08-18 17:05:36, 2010-11-25 17:45:31, 2011-02-27 16:58:20, 2011-05-07 14:53:45, 2011-07-23 14:42:36, 2011-09-10 00:47:25, 2011-09-10 22:48:43, 2011-11-24 15:08:04, 2011-12-04 17:12:11, 2011-12-11 17:42:30, 2012-01-07 01:55:08, 2012-01-27 17:25:35, 2012-02-02 23:27:25, 2012-02-08 02:42:30, 2012-02-12 02:12:58, 2012-02-19 20:00:18, 2012-03-01 21:38:08, 2012-03-03 01:36:16, 2012-03-04 19:02:10, 2012-03-11 21:45:52, 2012-03-27 22:43:44, 2012-03-31 22:49:05, 2012-04-17 22:28:54, 2012-04-21 21:23:45, 2012-06-17 21:59:17, 2012-07-07 21:11:44, 2012-07-10 20:41:38, 2012-07-13 00:16:29, 2012-07-15 16:47:10, 2012-07-19 18:19:28, 2012-07-22 20:14:42, 2012-07-29 15:35:48, 2012-08-15 18:05:50, 2012-08-23 23:39:12, 2012-08-29 11:11:15, 2012-09-08 14:56:13, 2012-09-16 15:30:07, 2012-09-17 21:54:50...",334
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}",5,None,None,"2019-06-28 21:35:09, 2019-09-12 21:07:48, 2019-11-09 23:28:44, 2020-02-15 21:10:58, 2020-08-02 16:05:42, 2020-09-12 19:47:26, 2020-11-21 17:15:26, 2020-12-05 20:55:53, 2021-03-27 16:33:48, 2021-05-01 16:10:04, 2021-06-19 18:40:25, 2021-08-20 22:20:37, 2021-09-10 22:26:40, 2021-12-17 21:55:20",13
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'OutdoorSeating': 'True', 'RestaurantsReservations': 'False', 'Caters': 'False', 'RestaurantsTakeOut': 'True', 'Alcohol': 'u'none'', 'Ambience': 'None', 'GoodForKids': 'True', 'RestaurantsPriceRange2': '1', 'ByAppointmentOnly': 'False', 'CoatCheck': 'False', 'DogsAllowed': 'False', 'RestaurantsTableService': 'False', 'RestaurantsGoodForGroups': 'True', 'RestaurantsDelivery': 'True', 'WiFi': 'u'no'', 'WheelchairAccessible': 'True', 'HasTV': 'True', 'HappyHour': 'False', 'DriveThru': 'True', 'BikeParking': 'False'}","Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturday': '9:0-22:0', 'Sunday': '8:0-22:0'}",9,1,Low,"2012-12-16 05:27:33, 2013-02-15 04:00:46, 2013-02-18 00:43:56, 2013-04-26 23:57:46, 2013-06-26 22:01:59, 2013-10-26 00:59:54, 2015-09-20 16:54:57, 2017-04-23 04:50:47, 2017-05-20 23:56:27, 2017-07-12 02:13:44, 2018-11-20 04:02:12, 2019-08-03 01:17:44, 2020-02-14 17:12:41, 2020-08-16 18:45:56, 2020-09-05 20:16:58, 2021-01-24 20:15:36, 2021-04-14 17:18:59, 2021-06-28 16:45:40, 2021-07-16 01:25:24, 2021-09-17 17:06:57, 2021-12-12 02:25:20, 202

2. Checkin dataset with total count information


In [23]:
df_checkin.head()

,business_id,date,checkin_count
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020-07-24 22:42:27, 2020-10-24 21:36:13, 2020-12-09 21:23:33, 2021-01-20 17:34:57, 2021-04-30 21:02:03, 2021-05-25 21:16:54, 2021-08-06 21:08:08, 2021-10-02 15:15:42, 2021-11-11 16:23:50",10
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011-07-18 22:30:31, 2012-09-07 20:28:50, 2013-03-27 15:57:36, 2013-08-13 00:31:34, 2013-08-13 00:31:48, 2013-09-23 17:39:38, 2013-11-18 06:34:08, 2014-04-12 23:04:47",9
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22",1
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012-03-11 10:30:02, 2012-04-24 07:07:59, 2012-04-24 07:43:31, 2013-05-25 16:41:10, 2014-05-02 15:49:55, 2014-09-18 02:28:23, 2014-11-10 15:16:43, 2015-09-27 13:18:32",9
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014-09-30 14:41:47, 2014-10-23 18:22:28, 2015-04-27 19:55:00, 2015-09-21 12:52:09, 2015-10-01 12:46:16, 2015-10-22 13:35:04, 2016-01-14 12:27:43, 2016-02-01 15:15:07, 2016-02-11 12:22:47, 2016-03-31 23:15:46, 2016-04-11 13:11:34, 2016-05-25 12:44:11, 2016-06-27 15:46:11, 2016-06-30 15:14:28, 2016-07-28 14:15:59, 2016-09-22 20:05:06, 2016-11-16 19:00:11, 2016-12-27 14:36:14, 2017-01-24 20:18:16, 2017-02-21 16:01:49, 2017-03-21 14:08:44, 2017-05-21 16:07:21, 2017-08-07 14:17:39, 2021-06-21 19:59:50",25


3. Tip information with the year extracted out of date the tip was created

In [24]:
tip.head(10)

,user_id,business_id,text,date,compliment_count,year_Tip
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0,2012
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban sandwiches,2013-02-05 18:35:10,0,2013
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0,2013
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0,2017
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0,2012
5,trf3Qcz8qvCDKXiTgjUcEg,7Rm9Ba50bw23KTA8RedZYg,"Chili Cup + Single Cheeseburger with onion, pickle, and relish + Vanilla Coca-Cola...so far.",2012-03-13 04:00:52,0,2012
6,SMGAlRjyfuYu-c-22zIyOg,kH-0iXqkL7b8UXNpguBMKg,"Saturday, Dec 7th 2013, ride Patco's Silver Sleigh w/ Santa & his elves on a decorated train into Center City. Trains leave from Lindenwold at 10am, 11:15am, & 12:30pm, and make all stops. Great for kids!",2013-12-03 23:42:15,0,2013
7,YVBB9g23nuVJ0u44zK0pSA,jtri188kuhe_AuEOJ51U_A,This is probably the best place in the cool Springs area to watch a game and eat,2016-11-22 22:14:58,0,2016
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0,2012
9,4ay-fdVks5WMerYL_htkGQ,pICJRcyqW1cF96Q3XhLSbw,Starbucks substitute in boring downtown Tampa. Ugh. Never again!,2012-06-09 22:57:04,0,2012


### Extract to delimited file

In [29]:
tip.to_csv('tip_m2.txt', sep='|', index=False)


In [25]:

bus_rest_merge.to_csv('bus_rest_merge_m2.txt', sep='|', index=False)